In [1]:
import sys
sys.path.append('..') #to add top-level to path

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import r2_score, accuracy_score, f1_score

from pandas.plotting import scatter_matrix
#import seaborn as sns

import warnings
warnings.filterwarnings('ignore')


from modules.project_helper import VolFeatures, FuturesCloseData

In [2]:
import pickle
file = open("../data/features/full_features.pkl",'rb')
full_features = pickle.load(file)
file.close()

In [3]:
instrument_list = ['ES', 'NQ', 'CD', 'EC', 'JY', 'MP', 'TY', 'US', 'C', 'S', 'W', 'CL', 'GC']
x_dict={}
y_dict={}
y_returns={}
for inst in instrument_list:
    #y_dict[inst] = 2 * (full_features[inst][inst] >=0)- 1
    y_dict[inst] = (full_features[inst][inst]>=0).astype(int)
    x_dict[inst] = full_features[inst].drop([inst], axis=1)
    returns = full_features[inst][inst]
    y_returns[inst] = returns[[i in x_dict[inst].index for i in returns.index]]
    if sum(y_returns[inst].index != x_dict[inst].index)!=0:
        raise Exception('Returns and X indices dont match')


In [4]:
class MLModel:
    def __init__(self,model,inst,x_dict,y_dict,y_returns,hyper_parameters={}):
        self.inst = inst
        self.x = x_dict[inst]
        self.y = y_dict[inst]
        self.y_returns = y_returns[inst]
        self.hyper_parameters=hyper_parameters

        self.model = model
        self.accuracy_train = None
        self.accuracy_test = None
        self.sharpe = None
        self.f1_test = None
        self.prediction_ratio = None
        
        self.strat_rets = None
        self.strat_rets_cum = None
        
    
        self.train_predictions = None
        self.test_predictions = None
        self.position = None
        
        self.train_class_balance = None
        self.test_class_balance = None
    
    def split_data(self):
        self.X_train, \
        self.X_test, \
        self.y_train, \
        self.y_test,\
        self.y_returns_train,\
        self.y_returns_test = train_test_split(self.x, self.y, self.y_returns, test_size=0.20, shuffle=False)


    def train_model(self):
        #self.model = OLS(self.y_train, self.X_train)
        #self.model = self.model.fit()

        self.model = self.model(random_state=0,**self.hyper_parameters).fit(self.X_train,self.y_train)

    def evaluate_sharpe(self, cutoff=0.50):
        rets = self.strategy_returns(cutoff)[0]
        self.sharpe = np.sqrt(252)*np.mean(rets) / np.std(rets)
    
    def get_position(self, cutoff=0.50):
        # converting predictions from {0,1} to {-1,1}, short/long
        self.position = 2 * self.model.predict(self.X_test) - 1
        self.position[self.model.predict_proba(self.X_test).max(axis=1) <= cutoff] = 0
        return self.position
    
    def strategy_returns(self,cutoff=0.50):
        x = self.get_position(cutoff=cutoff)[:-1]
        y = self.y_returns_test[:-1] #make sure returns are logs
        self.strat_rets = x * y
        self.strat_rets_cum =  self.strat_rets.cumsum()
        return self.strat_rets, self.strat_rets_cum


    def evaluate_model(self):
        self.accuracy_train = self.model.score(self.X_train, self.y_train)
        self.accuracy_test = self.model.score(self.X_test, self.y_test)
        self.f1_test = f1_score(self.y_test,self.test_predictions)
        self.evaluate_sharpe()
        self.prediction_ratio = np.mean(self.test_predictions)
    
    def generate_predictions(self):
        self.train_predictions = self.model.predict(self.X_train)
        self.test_predictions = self.model.predict(self.X_test)
        
        self.train_class_balance = np.mean(self.train_predictions)
        self.test_class_balance = np.mean(self.test_predictions)
    



        
class AssetModels:
    def __init__(self,inst,x_dict,y_dict,y_returns,hyper_parameters):
        #self.model = None
        self.logistic_model = MLModel(LogisticRegression,inst,x_dict,y_dict,y_returns,
                                     hyper_parameters.get('logistic') if hyper_parameters.get('logistic') else {})
        self.rf_model = MLModel(RandomForestClassifier,inst,x_dict,y_dict,y_returns,
                                     hyper_parameters.get('rf') if hyper_parameters.get('rf') else {})
        self.tree_model = MLModel(DecisionTreeClassifier,inst,x_dict,y_dict,y_returns,
                                     hyper_parameters.get('tree') if hyper_parameters.get('tree') else {})
        self.boosted_tree_model = MLModel(GradientBoostingClassifier,inst,x_dict,y_dict,y_returns,
                                     hyper_parameters.get('boosted_tree') if hyper_parameters.get('boosted_tree') else {})

        self.ml_models = {'logistic':self.logistic_model,
                          'rf':self.rf_model,
                          #'tree':self.tree_model,
                          'boosted_tree':self.boosted_tree_model
                         }
        
        self.best_model_name = None
        self.best_model = None
        
        self.best_model_accuracy = None
        self.best_model_sharpe = None

        self.accuracies_train = None
        self.accuracies_test = None
        self.sharpe_values = None
        self.f1_scores = None
        self.prediction_ratios = None
        
    def get_best_model(self):
        self.accuracies_test = pd.DataFrame.from_dict(
            {model_name:model.accuracy_test for model_name,model in self.ml_models.items()},
            orient='index',columns=['test_accuracy']
        )
        self.accuracies_train = pd.DataFrame.from_dict(
            {model_name:model.accuracy_train for model_name,model in self.ml_models.items()},
            orient='index',columns=['test_accuracy']
        )
        
        self.sharpe_values = pd.DataFrame.from_dict(
            {model_name:model.sharpe for model_name,model in self.ml_models.items()},
            orient='index',columns=['sharpe']
        )
        
        self.f1_scores = pd.DataFrame.from_dict(
            {model_name:model.f1_test for model_name,model in self.ml_models.items()},
            orient='index',columns=['f1_score']
        )
        
        self.prediction_ratios = pd.DataFrame.from_dict(
            {model_name:model.prediction_ratio for model_name,model in self.ml_models.items()},
            orient='index',columns=['prediction_ratio']
        )



        
        self.best_model_name = self.sharpe_values.idxmax().tolist()[0]
        self.best_model = self.ml_models.get(self.best_model_name)
        
        self.best_model_accuracy = self.accuracies_test[self.accuracies_test.index==self.best_model_name]
        self.best_model_accuracy.index.name = 'best_model'
        self.best_model_accuracy.reset_index(inplace=True)
        
        self.best_model_sharpe = self.sharpe_values[self.sharpe_values.index==self.best_model_name]
        self.best_model_f1 = self.f1_scores[self.f1_scores.index==self.best_model_name]
        self.best_model_prediction_ratio = self.prediction_ratios[self.prediction_ratios.index==self.best_model_name]

        #self.best_model_sharpe.index.name = 'best_model'
        #self.best_model_sharpe.reset_index(inplace=True)

        
        
    def run(self):
        {model.split_data() for model in self.ml_models.values()}
        {model.train_model() for model in self.ml_models.values()}
        {model.generate_predictions() for model in self.ml_models.values()}
        {model.evaluate_model() for model in self.ml_models.values()}
        self.get_best_model()

        
class ModelBuildier:
    def __init__(self,x_dict,y_dict,y_returns,instrument_list,hyper_parameters={}):
        self.x_dict = x_dict
        self.y_dict = y_dict
        self.hyper_parameters = hyper_parameters
        self.instrument_list = instrument_list
        self.asset_models =  {inst: AssetModels(inst,x_dict,y_dict,y_returns,
                            hyper_parameters.get(inst) if hyper_parameters.get(inst) else {})\
                              for inst in instrument_list}
        
        self.accuracies_best = pd.DataFrame()
        self.accuracies_all = pd.DataFrame()
        
    def get_accuracies(self):
        for inst in instrument_list:
            accuracy_df = self.asset_models[inst].best_model_accuracy
            accuracy_df.index = [inst]
            sharpe_df = self.asset_models[inst].best_model_sharpe
            sharpe_df.index = [inst]
            f1_df = self.asset_models[inst].best_model_f1
            f1_df.index = [inst]
            prediction_ratio_df = self.asset_models[inst].best_model_prediction_ratio
            prediction_ratio_df.index = [inst]


            accuracy_df = accuracy_df.join(sharpe_df).join(f1_df).join(prediction_ratio_df)
            self.accuracies_best = self.accuracies_best.append(accuracy_df)
            
            all_accuracy_df = self.asset_models[inst].accuracies_test
            all_sharpe_df = self.asset_models[inst].sharpe_values
            all_f1_df = self.asset_models[inst].f1_scores
            all_prediction_ratio_df = self.asset_models[inst].prediction_ratios


            all_accuracy_df = all_accuracy_df.join(all_sharpe_df).join(all_f1_df).join(all_prediction_ratio_df)
            all_accuracy_df.index.name = 'model'
            all_accuracy_df = all_accuracy_df.reset_index()
            all_accuracy_df['asset'] = inst
            self.accuracies_all = self.accuracies_all.append(all_accuracy_df)
        self.accuracies_all = self.accuracies_all.set_index('asset')

            
            
        
    
    def run(self):
        {inst: model.run() for inst,model in self.asset_models.items()}
        self.get_accuracies()
        
        

In [5]:
hp = {
    'ES':{
        'logistic':{
            'C':0.1
        },
        'rf':{
            'max_depth':4,
            'max_features':13,               
        },
        'boosted_tree':{
            'max_depth':6,
            'max_features':16,               
        },
    },
    'NQ':{
        'logistic':{
            'C':0.01
        },
        'rf':{
            'max_depth':4,
            'max_features':3,               
        },
        'boosted_tree':{
            'max_depth':14,
            'max_features':5,               
        },
    },   
    'CD':{
        'logistic':{
            'C':0.01
        },
        'rf':{
            'max_depth':13,
            'max_features':6,               
        },
        'boosted_tree':{
            'max_depth':8,
            'max_features':6,               
        },
    },
    'EC':{
        'logistic':{
            'C':1
        },
        'rf':{
            'max_depth':12,
            'max_features':6,               
        },
        'boosted_tree':{
            'max_depth':11,
            'max_features':3,               
        },
    },
    'JY':{
        'logistic':{
            'C':0.001
        },
        'rf':{
            'max_depth':4,
            'max_features':22,               
        },
        'boosted_tree':{
            'max_depth':13,
            'max_features':16,               
        },
    },
    'MP':{
        'logistic':{
            'C':10000
        },
        'rf':{
            'max_depth':8,
            'max_features':19,               
        },
        'boosted_tree':{
            'max_depth':11,
            'max_features':22,               
        },
    },
    'TY':{
        'logistic':{
            'C':0.001
        },
        'rf':{
            'max_depth':7,
            'max_features':13,               
        },
        'boosted_tree':{
            'max_depth':8,
            'max_features':16,               
        },
    },
    'US':{
        'logistic':{
            'C':0.001
        },
        'rf':{
            'max_depth':5,
            'max_features':3,               
        },
        'boosted_tree':{
            'max_depth':11,
            'max_features':16,               
        },
    },
    'C':{
        'logistic':{
            'C':0.01
        },
        'rf':{
            'max_depth':8,
            'max_features':19,               
        },
        'boosted_tree':{
            'max_depth':11,
            'max_features':5,               
        },
    },
    'S':{
        'logistic':{
            'C':1
        },
        'rf':{
            'max_depth':13,
            'max_features':10,               
        },
        'boosted_tree':{
            'max_depth':6,
            'max_features':10,               
        }, 
    },
    'W':{
        'logistic':{
            'C':10,
        },
        'rf':{
            'max_depth':4,
            'max_features':3,               
        },
        'boosted_tree':{
            'max_depth':12,
            'max_features':6,               
        },
    },
    'CL':{
        'logistic':{
            'C':0.1
        },
        'rf':{
            'max_depth':4,
            'max_features':4,               
        },
        'boosted_tree':{
            'max_depth':5,
            'max_features':22,               
        },
    },
    'GC':{
        'logistic':{
            'C':10000,
        },
        'rf':{
            'max_depth':4,
            'max_features':3,               
        },
        'boosted_tree':{
            'max_depth':11,
            'max_features':25,               
        }
        
    }
}
    

    


In [6]:
model_builder = ModelBuildier(x_dict,y_dict,y_returns,instrument_list,hyper_parameters=hp)
model_builder.run()

## Best Model Metrics

In [7]:
model_builder.accuracies_best

,best_model,test_accuracy,sharpe,f1_score,prediction_ratio
ES,logistic,0.557143,0.910835,0.710280,0.978571
NQ,logistic,0.542857,0.443255,0.703704,1.000000
CD,boosted_tree,0.550000,3.347600,0.442478,0.278571
EC,logistic,0.492857,0.013225,0.236559,0.185714
JY,rf,0.521429,0.521220,0.192771,0.092857
MP,rf,0.535714,1.480037,0.591195,0.607143
TY,logistic,0.507143,0.811234,0.591716,0.621429
US,logistic,0.542857,1.231922,0.648352,0.714286
C,rf,0.500000,0.760028,0.513889,0.521429
S,boosted_tree,0.528571,0.867972,0.597561,0.664286


# All Model Metrics

In [8]:
model_builder.accuracies_all

,model,test_accuracy,sharpe,f1_score,prediction_ratio
asset,,,,,
ES,logistic,0.557143,0.910835,0.710280,0.978571
ES,rf,0.557143,0.894590,0.696078,0.907143
ES,boosted_tree,0.528571,-0.574667,0.616279,0.678571
NQ,logistic,0.542857,0.443255,0.703704,1.000000
NQ,rf,0.521429,0.254548,0.685446,0.978571
NQ,boosted_tree,0.471429,-0.594506,0.622449,0.857143
CD,logistic,0.478571,-0.093572,0.075949,0.035714
CD,rf,0.442857,-1.501671,0.290909,0.257143
CD,boosted_tree,0.550000,3.347600,0.442478,0.278571


In [9]:
def feat_imp(inst):
    try: df = pd.DataFrame(model_builder.asset_models[inst].best_model.model.feature_importances_, 
             index = x_dict[inst].columns, 
             columns=['feature_importance'] ).sort_values(by='feature_importance', ascending=False).head(10)
    except:
         df = pd.DataFrame(model_builder.asset_models[inst].ml_models['rf'].model.feature_importances_, 
             index = x_dict[inst].columns, 
             columns=['feature_importance'] ).sort_values(by='feature_importance', ascending=False).head(10)
    return df

In [10]:
feat_imp('ES')

,feature_importance
ES_max_tweet,0.074269
intra_diff_15_5,0.070397
intra_ret_1,0.061047
negative_proportion_mean,0.058514
neutral_proportion_mean,0.052762
ES_volume_chg,0.047888
combined_score_min,0.047106
ES_1M_RR25,0.039551
positive_proportion_min,0.035958
intra_ret_5,0.034167


In [11]:
feat_imp('NQ')

,feature_importance
NQ_min_tweet,0.066365
combined_score_max,0.052120
intra_ret_5,0.050224
positive_proportion_min,0.049432
NQ_volume_chg,0.040516
negative_proportion_mean,0.040422
combined_score_min,0.039296
intra_diff_15_5,0.035559
positive_proportion_max,0.034388
topic_16,0.033091


In [12]:
feat_imp('CD')

,feature_importance
intra_ret_15,0.046800
neutral_proportion_mean,0.036919
CD_1M_atm_vol,0.036133
combined_score_max,0.035240
CD_2M_1M_atm_vol,0.035019
intra_ret_1,0.033397
combined_score_min,0.032209
intra_diff_15_5,0.031621
CD_2M_Fly25,0.031292
positive_proportion_max,0.031173


In [13]:
feat_imp('EC')

,feature_importance
EC_2M_1M_atm_vol,0.057560
EC_1M_RR25,0.047242
EC_1M_atm_vol,0.040735
intra_ret_1,0.032941
positive_proportion_mean,0.031785
EC_max_tweet,0.031686
intra_diff_15_5,0.031079
combined_score_min,0.031041
EC_min_tweet,0.031006
negative_proportion_max,0.030417


In [14]:
feat_imp('JY')

,feature_importance
JY_max_tweet,0.106108
JY_1M_RR25,0.070245
neutral_proportion_min,0.066852
intra_diff_15_5,0.047666
combined_score_mean,0.046638
JY_1M_atm_vol,0.045104
neutral_proportion_mean,0.043615
intra_ret_5,0.039528
topic_21,0.037486
JY_2M_RR25,0.037304


In [15]:
feat_imp('MP')

,feature_importance
MP_2M_1M_atm_vol,0.068118
MP_daily_tweet,0.049691
intra_ret_15,0.048185
MP_volume_chg,0.047118
MP_2M_RR25,0.040110
MP_1M_atm_vol,0.039807
MP_2M_Fly25,0.033880
negative_proportion_mean,0.031641
MP_1M_Fly25,0.031041
neutral_proportion_min,0.030812


In [16]:
feat_imp('TY')

,feature_importance
TY_1M_atm_vol,0.060961
TY_max_tweet,0.053037
intra_ret_1,0.051573
combined_score_min,0.045237
negative_proportion_max,0.043923
intra_ret_15,0.040154
combined_score_max,0.039107
intra_ret_5,0.038404
TY_2M_1M_atm_vol,0.034164
intra_diff_15_5,0.033487


In [17]:
feat_imp('US')

,feature_importance
positive_proportion_min,0.070089
combined_score_max,0.069649
neutral_proportion_max,0.057495
neutral_proportion_mean,0.045441
US_min_tweet,0.039443
topic_2,0.038052
US_volume_chg,0.036733
US_2M_Fly25,0.035131
combined_score_mean,0.033994
combined_score_min,0.030817


In [18]:
feat_imp('C')

,feature_importance
C_2M_RR25,0.061408
C_volume_chg,0.052950
intra_diff_15_5,0.048167
neutral_proportion_mean,0.044028
C_2M_Fly25,0.039000
C_min_tweet,0.036030
intra_ret_15,0.030733
C_1M_atm_vol,0.030515
positive_proportion_mean,0.028637
C_1M_RR25,0.027340


In [19]:
feat_imp('C')

,feature_importance
C_2M_RR25,0.061408
C_volume_chg,0.052950
intra_diff_15_5,0.048167
neutral_proportion_mean,0.044028
C_2M_Fly25,0.039000
C_min_tweet,0.036030
intra_ret_15,0.030733
C_1M_atm_vol,0.030515
positive_proportion_mean,0.028637
C_1M_RR25,0.027340


In [20]:
feat_imp('S')

,feature_importance
S_daily_tweet,0.041264
S_volume_chg,0.039593
S_max_tweet,0.036727
S_1M_Fly25,0.036214
intra_diff_15_5,0.033072
S_2M_Fly25,0.032615
combined_score_min,0.030162
S_down_gamma,0.029689
neutral_proportion_mean,0.028666
S_1M_atm_vol,0.028151


In [21]:
feat_imp('W')

,feature_importance
W_volume_chg,0.044661
W_daily_tweet,0.037529
neutral_proportion_max,0.031992
W_max_tweet,0.028864
intra_diff_15_5,0.027761
W_2M_RR25,0.027463
W_2M_1M_atm_vol,0.026719
W_1M_atm_vol,0.026689
neutral_proportion_min,0.026553
negative_proportion_mean,0.026541


In [22]:
feat_imp('GC')

,feature_importance
GC_2M_RR25,0.083589
neutral_proportion_min,0.055835
combined_score_min,0.051325
GC_2M_1M_atm_vol,0.050134
GC_2M_Fly25,0.048297
GC_max_tweet,0.047774
intra_diff_15_5,0.045096
negative_proportion_min,0.038150
positive_proportion_mean,0.036196
intra_ret_5,0.036186
